In [1]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
#%config InlineBackend.figure_format = 'svg'

os.chdir('..') #change to parent directory

from src.data_access import (
    load_scraped_data,
)

from src.data_processing import (
    merge_scraped_dataframes,
    process_nans,
    rename_columns,
    extract_new_columns,
    reorder_columns,
)

from src.webscraping.utils import (
    validate_scraped_dataframes,
)

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')


ImportError: cannot import name 'load_scraped_data' from 'src.data_access' (e:\OneDrive\nba_analysis_project\src\data_access\__init__.py)

In [2]:
# retrieve the scraped data and merge into a single joined dataframe

scraped_data = [] #list of dataframes
scraped_data = load_scraped_data()
if len(scraped_data) == 0:
    print('No data loaded')
else:
    if validate_scraped_dataframes(scraped_data):
        games = merge_scraped_dataframes(scraped_data)
        games.head()
    else:
        print("validation failed")
        

e:\OneDrive\nba_analysis_project\src\data_access.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATAPATH / file)


Dataframe 1 does not match the game ids of the first dataframe


In [ ]:
games.info()

In [ ]:

games = pd.read_csv(DATAPATH / "games.csv")
games.head()

In [ ]:
games.info()

In [ ]:
# GAME_ID and TEAM_ID are the primary keys, so let's put them up front

col = games.pop("TEAM_ID")
games.insert(0, "TEAM_ID", col)

col = games.pop("GAME_ID")
games.insert(0, "GAME_ID", col)

games.info()


In [ ]:
# The "Match Up" column indicates which team is the home team and which is the visitor, but let's explicitly flag the home team.
# If the "Match Up" column contains "@" then the team is the visitor, if it contains "vs." then the team is the home team.

games["HOME_TEAM"] = games["Match Up"].str.contains("vs.").astype(int)

In [ ]:
# Let's move HOME_TEAM to the 3rd column

col = games.pop("HOME_TEAM")
games.insert(2, "HOME_TEAM", col)

games.info()

In [ ]:
# Convert W/L column to integer

games["WIN"] = games["W/L"].str.contains("W").astype(int)

# Let's move WIN to the 4th column

col = games.pop("WIN")
games.insert(3, "WIN", col)

# Drop the W/L column

games = games.drop("W/L", axis=1)

games.head()

In [ ]:
# Minutes played is not really that useful of a feature in terms of a an overall team evaluation metric. It is more of a player metric. 
# It might be useful in comparing the stats from a regular length game to an overtime game, but a simple flag for overtime might be good enough.

# Let's see how many games went into overtime (MIN > 240) 

overtimegames = games[games["MIN"] > 240].shape[0]
totalgames = games.shape[0]

print(f"Number of overtime games: {overtimegames} out of {totalgames} total games ({overtimegames/totalgames:.2%})")


In [ ]:
# About 6% of games went into overtime. Not a large amount, but not super small amount either, so we will both keep MIN and set a flag for overtime games
# This will give us more flexibility in the future since we may or may not want to include MIN in our analysis.

games["OVERTIME"] = (games["MIN"] > 240).astype(int)

# Let's move OVERTIME to the 7th column to keep it next to MIN

col = games.pop("OVERTIME")
games.insert(7, "OVERTIME", col)


games.head()


In [ ]:
# The first digit of the GAME_ID denotes whether the game was played in the regular season (2) or the playoffs (4) or play-in (5)
# The second and third digits denote the season (e.g. 21 for the 2021-2022 season)

# To make it easier to extract this info, first let's convert GAME_ID to a string

games["GAME_ID"] = games["GAME_ID"].astype(str)

games["SEASON"] = games["GAME_ID"].str[1:3].astype(int) + 2000

# create flag for playoff games where the first digit of the GAME_ID is greater than 2
games["PLAYOFF"] = (games["GAME_ID"].str[0].astype(int) > 2).astype(int)

# Let's move SEASON to the 5th column and PLAYOFF to the 6th column

col = games.pop("SEASON")
games.insert(5, "SEASON", col)

col = games.pop("PLAYOFF")
games.insert(6, "PLAYOFF", col)

games.head()



In [ ]:
# Now group the columns - first the game info, then the team info, then the game stats for that team
all_columns = games.columns.tolist()

game_info = ["GAME_ID", "SEASON", "Game Date", "PLAYOFF", "OVERTIME", "MIN",]
team_info = ["TEAM_ID", "HOME_TEAM", "Team", "Match Up"] 
team_stats = [col for col in all_columns if col not in game_info + team_info]

games = games[game_info + team_info + team_stats]

games.head()


In [ ]:
# save the cleaned data
games.to_csv(DATAPATH / "games_cleaned.csv", index=False)
